# 🇪🇹 Ethiopian Food Recommendation System
## Advanced ML Project - Weighted KNN for BMI-based Diet Recommendations

This notebook demonstrates a complete data science workflow including:
- Data Loading and Exploration
- Data Cleaning
- Data Integration
- Data Transformation
- Data Reduction
- Visualization
- Machine Learning (Weighted KNN)

# 1. Import Required Libraries

First, we need to import the necessary libraries for data analysis.

- **NumPy** - Scientific computing
- **Pandas** - Data manipulation
- **Matplotlib/Seaborn** - Visualization
- **Scikit-learn** - Machine learning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported successfully!')

Libraries imported successfully!


## Understanding Data Structures
- **Series:** One-dimensional labeled array
- **DataFrame:** Two-dimensional labeled data structure

In [2]:
# Series example
series_data = pd.Series([1, 2, 'A', True], index=range(1, 5))
series_data

1       1
2       2
3       A
4    True
dtype: object

In [3]:
# DataFrame example
data = {'Food': ['Doro Wot', 'Kitfo', 'Shiro'], 'Calories': [450, 350, 280]}
pd.DataFrame(data)

,Food,Calories
0,Doro Wot,450
1,Kitfo,350
2,Shiro,280


# 2. Load the Dataset

Load .csv data into dataframe using pandas `read_csv()` function.
Make sure the CSV file is in the correct directory!

In [4]:
# Load the Ethiopian foods dataset
data = pd.read_csv('data/ethiopian_foods.csv')
print(f'Dataset loaded: {len(data)} foods')
data

Dataset loaded: 75 foods


,Name,Category,MealTime,Calories,Protein,Carbs,Fat,Fiber,SpiceLevel,IsVegetarian,PrepTime,Description,Ingredients,BMICategory,Reason,Cuisine
0,Genfo,Breakfast,Breakfast,280,8,50,6,3,NaN,Yes,20,Thick porridge served with spiced butter,barley flour;water;kibbe;berbere,Underweight,High in carbs and healthy fats to help gain we...,Ethiopian
1,Chechebsa,Breakfast,Breakfast,350,8,42,16,3,Medium,Yes,25,Shredded flatbread with spiced butter,flour;kibbe;berbere;sugar,Underweight,Calorie-dense breakfast to support weight gain,Ethiopian
2,Fatira,Breakfast,Breakfast,380,10,55,14,2,NaN,Yes,30,Layered fried bread with honey,flour;eggs;butter;honey,Underweight,High calorie traditional bread for energy,Ethiopian
3,Kinche Be Kibbe,Breakfast,Breakfast,320,10,48,10,5,NaN,Yes,25,Cracked wheat with spiced butter,cracked wheat;kibbe;cardamom,Underweight,Nutritious whole grain with healthy fats,Ethiopian
4,Atmit,Breakfast,Breakfast,220,6,40,4,3,NaN,Yes,15,Thick oat porridge drink,oat flour;water;honey;milk,Underweight,Easy to digest calorie source for weight gain,Ethiopian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Banana Smoothie,Snack,Snack,280,8,50,6,4,NaN,Yes,5,Banana milk smoothie,banana;milk;honey;oats,Underweight,Easy calories for weight gain,International
71,Lebleb,Snack,Snack,160,9,22,4,6,Low,Yes,10,Roasted chickpeas snack,chickpeas;salt;spices,Overweight,High protein low fat healthy snack,Ethiopian
72,Apple Slices,Snack,Snack,60,0,15,0,3,NaN,Yes,5,Fresh apple slices,apple,Obese,Low calorie healthy snack,International
73,Carrot Sticks,Snack,Snack,40,1,9,0,3,NaN,Yes,5,Raw carrot sticks,carrots,Obese,Very low calorie crunchy snack,International


## 2.1 Explore the Data
Use various pandas functions to examine the structure, summary statistics, and sample records.

In [5]:
# Display first 5 records
data.head()

,Name,Category,MealTime,Calories,Protein,Carbs,Fat,Fiber,SpiceLevel,IsVegetarian,PrepTime,Description,Ingredients,BMICategory,Reason,Cuisine
0,Genfo,Breakfast,Breakfast,280,8,50,6,3,NaN,Yes,20,Thick porridge served with spiced butter,barley flour;water;kibbe;berbere,Underweight,High in carbs and healthy fats to help gain we...,Ethiopian
1,Chechebsa,Breakfast,Breakfast,350,8,42,16,3,Medium,Yes,25,Shredded flatbread with spiced butter,flour;kibbe;berbere;sugar,Underweight,Calorie-dense breakfast to support weight gain,Ethiopian
2,Fatira,Breakfast,Breakfast,380,10,55,14,2,NaN,Yes,30,Layered fried bread with honey,flour;eggs;butter;honey,Underweight,High calorie traditional bread for energy,Ethiopian
3,Kinche Be Kibbe,Breakfast,Breakfast,320,10,48,10,5,NaN,Yes,25,Cracked wheat with spiced butter,cracked wheat;kibbe;cardamom,Underweight,Nutritious whole grain with healthy fats,Ethiopian
4,Atmit,Breakfast,Breakfast,220,6,40,4,3,NaN,Yes,15,Thick oat porridge drink,oat flour;water;honey;milk,Underweight,Easy to digest calorie source for weight gain,Ethiopian


In [ ]:
# Display last 5 records
data.tail()

In [6]:
# Shape of data (rows, columns)
data.shape

(75, 16)

In [ ]:
# Size of the data
data.size

In [7]:
# Data info - columns, types, non-null counts
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          75 non-null     object
 1   Category      75 non-null     object
 2   MealTime      75 non-null     object
 3   Calories      75 non-null     int64 
 4   Protein       75 non-null     int64 
 5   Carbs         75 non-null     int64 
 6   Fat           75 non-null     int64 
 7   Fiber         75 non-null     int64 
 8   SpiceLevel    49 non-null     object
 9   IsVegetarian  75 non-null     object
 10  PrepTime      75 non-null     int64 
 11  Description   75 non-null     object
 12  Ingredients   75 non-null     object
 13  BMICategory   75 non-null     object
 14  Reason        75 non-null     object
 15  Cuisine       75 non-null     object
dtypes: int64(6), object(10)
memory usage: 9.5+ KB


In [ ]:
# Column names
data.columns

In [ ]:
# Columns as list
data.columns.tolist()

In [ ]:
# Get summary statistics for numerical columns
data.describe()

In [8]:
# Unique values in Cuisine column
data['Cuisine'].unique()

array(['Ethiopian', 'International'], dtype=object)

In [ ]:
# Unique values in BMICategory
data['BMICategory'].unique()

In [ ]:
# Unique values in MealTime
data['MealTime'].unique()

In [ ]:
# Mean value of Calories
data['Calories'].mean().round(2)

In [ ]:
# BMICategory value counts
data['BMICategory'].value_counts()

In [ ]:
# Cuisine value counts
data['Cuisine'].value_counts()

In [ ]:
# MealTime value counts
data['MealTime'].value_counts()

In [ ]:
# Filter specific columns
data[['Name', 'Calories', 'Protein', 'BMICategory']]

In [ ]:
# Filter Ethiopian foods with high calories
data[(data['Cuisine']=='Ethiopian') & (data['Calories'] > 300)]

In [ ]:
# Filter vegetarian foods for Obese category
data[(data['IsVegetarian']=='Yes') & (data['BMICategory']=='Obese')]

#### Writing Python Function to Calculate Statistics
Function to calculate mean, median and variance of Calories by BMI Category

In [ ]:
def calculate_nutrition_stats(df, bmi_category):
    '''Calculate mean, median and variance of Calories for a BMI category'''
    filtered = df[df['BMICategory'] == bmi_category]['Calories']
    return {
        'BMI Category': bmi_category,
        'Mean': filtered.mean().round(2),
        'Median': filtered.median(),
        'Variance': filtered.var().round(2)
    }

# Test the function
for cat in data['BMICategory'].unique():
    stats = calculate_nutrition_stats(data, cat)
    print(f"{stats['BMI Category']}: Mean={stats['Mean']}, Median={stats['Median']}, Var={stats['Variance']}")

# 3. Data Cleaning

Data cleaning involves:
- Handling missing values
- Removing duplicates
- Fixing data types
- Handling outliers

In [9]:
# Missing Values identification
print('Missing values per column:')
data.isnull().sum()

Missing values per column:


Name             0
Category         0
MealTime         0
Calories         0
Protein          0
Carbs            0
Fat              0
Fiber            0
SpiceLevel      26
IsVegetarian     0
PrepTime         0
Description      0
Ingredients      0
BMICategory      0
Reason           0
Cuisine          0
dtype: int64

In [ ]:
# Total missing values
print(f'Total missing values: {data.isnull().sum().sum()}')

### Imputation Methods
Common ways to handle missing values:
- Mean/Median/Mode Imputation
- Forward/Backward Fill
- Drop rows/columns

In [ ]:
# Example: Mean imputation for numerical columns
# df['column'].fillna(df['column'].mean(), inplace=True)

# Example: Mode imputation for categorical columns
# df['column'].fillna(df['column'].mode()[0], inplace=True)

# Our dataset has no missing values, but here's how you would handle them:
print('No missing values in our dataset - data is clean!')

In [ ]:
# Check for duplicates
print(f'Number of duplicate rows: {data.duplicated().sum()}')

In [ ]:
# Remove duplicates if any exist
# data.drop_duplicates(inplace=True)
print('No duplicates found in our dataset!')

# 4. Data Integration

Data integration combines data from multiple sources.
Methods include:
- **Concatenation** - Stacking dataframes
- **Merging** - Joining on common columns

#### Concat Example

In [ ]:
# Create sample dataframes to demonstrate concat
breakfast_foods = data[data['MealTime'] == 'Breakfast'].head(3)
lunch_foods = data[data['MealTime'] == 'Lunch'].head(3)

# Concatenate vertically
combined = pd.concat([breakfast_foods, lunch_foods], ignore_index=True)
print('Concatenated Breakfast and Lunch foods:')
combined[['Name', 'MealTime', 'Calories']]

#### Merging Example

In [ ]:
# Create a nutrition info dataframe
nutrition_info = pd.DataFrame({
    'BMICategory': ['Underweight', 'Normal', 'Overweight', 'Obese'],
    'DailyCalorieTarget': [2500, 2000, 1600, 1200],
    'Goal': ['Gain weight', 'Maintain', 'Lose gradually', 'Lose safely']
})

# Merge with main data
merged_data = pd.merge(data, nutrition_info, on='BMICategory', how='left')
merged_data[['Name', 'Calories', 'BMICategory', 'DailyCalorieTarget', 'Goal']].head(10)

# 5. Data Transformation

Data transformation includes:
- Renaming columns
- Encoding categorical variables
- Scaling/Normalizing numerical features
- Creating new features

In [10]:
# Create a copy for transformation
df = data.copy()
df.head()

,Name,Category,MealTime,Calories,Protein,Carbs,Fat,Fiber,SpiceLevel,IsVegetarian,PrepTime,Description,Ingredients,BMICategory,Reason,Cuisine
0,Genfo,Breakfast,Breakfast,280,8,50,6,3,NaN,Yes,20,Thick porridge served with spiced butter,barley flour;water;kibbe;berbere,Underweight,High in carbs and healthy fats to help gain we...,Ethiopian
1,Chechebsa,Breakfast,Breakfast,350,8,42,16,3,Medium,Yes,25,Shredded flatbread with spiced butter,flour;kibbe;berbere;sugar,Underweight,Calorie-dense breakfast to support weight gain,Ethiopian
2,Fatira,Breakfast,Breakfast,380,10,55,14,2,NaN,Yes,30,Layered fried bread with honey,flour;eggs;butter;honey,Underweight,High calorie traditional bread for energy,Ethiopian
3,Kinche Be Kibbe,Breakfast,Breakfast,320,10,48,10,5,NaN,Yes,25,Cracked wheat with spiced butter,cracked wheat;kibbe;cardamom,Underweight,Nutritious whole grain with healthy fats,Ethiopian
4,Atmit,Breakfast,Breakfast,220,6,40,4,3,NaN,Yes,15,Thick oat porridge drink,oat flour;water;honey;milk,Underweight,Easy to digest calorie source for weight gain,Ethiopian


In [ ]:
# Rename columns
df.rename(columns={'IsVegetarian': 'Vegetarian', 'PrepTime': 'PreparationTime'}, inplace=True)
df.columns.tolist()

### Label Encoding
Transform categorical variables to numerical values using LabelEncoder

In [ ]:
# Transform categorical columns using LabelEncoder
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

# Encode Cuisine column
df['Cuisine_Encoded'] = label.fit_transform(df['Cuisine'])
print('Cuisine encoding:')
print(dict(zip(label.classes_, range(len(label.classes_)))))

In [ ]:
# Encode BMICategory column
df['BMI_Encoded'] = label.fit_transform(df['BMICategory'])
print('BMICategory encoding:')
print(dict(zip(label.classes_, range(len(label.classes_)))))

In [ ]:
# Encode Vegetarian column
df['Vegetarian_Encoded'] = label.fit_transform(df['Vegetarian'])
print('Vegetarian encoding:')
print(dict(zip(label.classes_, range(len(label.classes_)))))

In [ ]:
# View transformed data
df[['Name', 'Cuisine', 'Cuisine_Encoded', 'BMICategory', 'BMI_Encoded', 'Vegetarian', 'Vegetarian_Encoded']].head(10)

### Feature Scaling
Normalize numerical features using StandardScaler

In [ ]:
# Scale numerical features
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
nutrition_cols = ['Calories', 'Protein', 'Carbs', 'Fat', 'Fiber']

# Create scaled versions
df_scaled = df.copy()
df_scaled[nutrition_cols] = scaler.fit_transform(df[nutrition_cols])

print('Original values:')
print(df[nutrition_cols].head())
print('\nScaled values (StandardScaler):')
print(df_scaled[nutrition_cols].head())

# 6. Data Reduction

Data reduction techniques:
- Removing unnecessary columns
- Dimensionality reduction
- Sampling

In [11]:
# Remove columns that won't be used in ML model
columns_to_drop = ['Description', 'Ingredients', 'Reason']
df_reduced = df.drop(columns_to_drop, axis=1)
print(f'Original columns: {len(df.columns)}')
print(f'After reduction: {len(df_reduced.columns)}')
df_reduced.columns.tolist()

Original columns: 16
After reduction: 13


['Name',
 'Category',
 'MealTime',
 'Calories',
 'Protein',
 'Carbs',
 'Fat',
 'Fiber',
 'SpiceLevel',
 'IsVegetarian',
 'PrepTime',
 'BMICategory',
 'Cuisine']

# 7. Visualization

Visualize the data to understand patterns and distributions.

In [ ]:
# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

In [ ]:
# Distribution of Calories by BMI Category
plt.figure(figsize=(10, 6))
sns.boxplot(x='BMICategory', y='Calories', data=data, order=['Underweight', 'Normal', 'Overweight', 'Obese'])
plt.title('Calories Distribution by BMI Category')
plt.xlabel('BMI Category')
plt.ylabel('Calories')
plt.show()

In [ ]:
# Count of foods by BMI Category
plt.figure(figsize=(8, 5))
sns.countplot(x='BMICategory', data=data, order=['Underweight', 'Normal', 'Overweight', 'Obese'])
plt.title('Number of Foods by BMI Category')
plt.xlabel('BMI Category')
plt.ylabel('Count')
plt.show()

In [ ]:
# Cuisine distribution
plt.figure(figsize=(8, 5))
data['Cuisine'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Foods by Cuisine')
plt.ylabel('')
plt.show()

In [ ]:
# Correlation heatmap for nutritional values
plt.figure(figsize=(8, 6))
nutrition_cols = ['Calories', 'Protein', 'Carbs', 'Fat', 'Fiber']
correlation = data[nutrition_cols].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Between Nutritional Values')
plt.show()

In [ ]:
# Scatter plot: Calories vs Protein colored by BMI Category
plt.figure(figsize=(10, 6))
colors = {'Underweight': 'blue', 'Normal': 'green', 'Overweight': 'orange', 'Obese': 'red'}
for cat in colors:
    subset = data[data['BMICategory'] == cat]
    plt.scatter(subset['Calories'], subset['Protein'], c=colors[cat], label=cat, alpha=0.7)
plt.xlabel('Calories')
plt.ylabel('Protein (g)')
plt.title('Calories vs Protein by BMI Category')
plt.legend()
plt.show()

In [ ]:
# Bar chart: Average Calories by Meal Time
plt.figure(figsize=(10, 5))
avg_calories = data.groupby('MealTime')['Calories'].mean().sort_values(ascending=False)
avg_calories.plot(kind='bar', color='teal')
plt.title('Average Calories by Meal Time')
plt.xlabel('Meal Time')
plt.ylabel('Average Calories')
plt.xticks(rotation=45)
plt.show()

# 8. Machine Learning

## Weighted K-Nearest Neighbors for Food Recommendation

Our ML approach:
1. Different BMI categories need different nutrition priorities
2. We assign different weights to features based on BMI
3. KNN finds foods closest to ideal nutrition targets

### BMI Calculation Functions

In [12]:
def calculate_bmi(weight, height):
    '''Calculate BMI from weight (kg) and height (cm)'''
    height_m = height / 100
    bmi = weight / (height_m ** 2)
    return round(bmi, 1)

def get_bmi_category(bmi):
    '''Get BMI category'''
    if bmi < 18.5:
        return 'Underweight', '🔵', 'You need to gain weight healthily'
    elif 18.5 <= bmi < 25:
        return 'Normal', '🟢', 'Maintain your healthy weight'
    elif 25 <= bmi < 30:
        return 'Overweight', '🟡', 'Focus on gradual weight loss'
    else:
        return 'Obese', '🔴', 'Prioritize weight loss for better health'

def calculate_daily_calories(weight, height, age, gender, bmi_category):
    '''Calculate recommended daily calories using Mifflin-St Jeor equation'''
    if gender == 'Male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    
    maintenance = bmr * 1.55
    
    if bmi_category == 'Underweight':
        return round(maintenance + 500), 'gain weight'
    elif bmi_category == 'Normal':
        return round(maintenance), 'maintain weight'
    elif bmi_category == 'Overweight':
        return round(maintenance - 400), 'lose weight gradually'
    else:
        return round(maintenance - 600), 'lose weight safely'

print('BMI functions defined!')

BMI functions defined!


### Feature Weights by BMI Category

In [ ]:
def get_bmi_weights(bmi_category):
    '''
    Get feature weights based on BMI category
    Weights: [Calories, Protein, Carbs, Fat, Fiber]
    '''
    if bmi_category == 'Underweight':
        return np.array([3.0, 1.5, 2.0, 2.5, 0.5])  # High calories priority
    elif bmi_category == 'Normal':
        return np.array([1.0, 1.5, 1.0, 1.0, 1.0])  # Balanced
    elif bmi_category == 'Overweight':
        return np.array([2.0, 2.0, 1.0, 1.5, 2.5])  # Low cal, high fiber
    else:  # Obese
        return np.array([3.5, 1.5, 1.0, 2.0, 3.0])  # Lowest cal, highest fiber

def get_target_nutrition(bmi_category):
    '''Get ideal target nutrition per meal based on BMI'''
    if bmi_category == 'Underweight':
        return [400, 28, 50, 20, 4]
    elif bmi_category == 'Normal':
        return [280, 22, 35, 12, 6]
    elif bmi_category == 'Overweight':
        return [180, 20, 25, 6, 8]
    else:
        return [120, 15, 15, 4, 6]

# Display weights table
weights_df = pd.DataFrame({
    'BMI Category': ['Underweight', 'Normal', 'Overweight', 'Obese'],
    'Calories': [3.0, 1.0, 2.0, 3.5],
    'Protein': [1.5, 1.5, 2.0, 1.5],
    'Carbs': [2.0, 1.0, 1.0, 1.0],
    'Fat': [2.5, 1.0, 1.5, 2.0],
    'Fiber': [0.5, 1.0, 2.5, 3.0]
})
print('Feature Weights by BMI Category:')
weights_df

### Weighted KNN Recommendation Model

In [ ]:
def weighted_knn_recommend(df, bmi_category, meal_time, n_recommendations=3):
    '''
    Advanced KNN with weighted features based on BMI category
    '''
    # Filter by meal time
    meal_foods = df[(df['MealTime'] == meal_time) | (df['MealTime'] == 'All')].copy()
    
    # Filter by BMI category
    bmi_foods = meal_foods[meal_foods['BMICategory'] == bmi_category]
    
    # Add Normal category if not enough foods
    if len(bmi_foods) < n_recommendations:
        normal_foods = meal_foods[meal_foods['BMICategory'] == 'Normal']
        bmi_foods = pd.concat([bmi_foods, normal_foods]).drop_duplicates()
    
    if len(bmi_foods) == 0:
        return None
    
    # Feature columns
    nutrition_cols = ['Calories', 'Protein', 'Carbs', 'Fat', 'Fiber']
    X = bmi_foods[nutrition_cols].values.astype(float)
    
    # Get BMI-specific weights
    weights = get_bmi_weights(bmi_category)
    
    # Scale the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Apply weights (KEY STEP!)
    X_weighted = X_scaled * weights
    
    # Create KNN model
    n_neighbors = min(n_recommendations, len(bmi_foods))
    model = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
    model.fit(X_weighted)
    
    # Get target and transform
    target = np.array([get_target_nutrition(bmi_category)])
    target_scaled = scaler.transform(target)
    target_weighted = target_scaled * weights
    
    # Find nearest neighbors
    distances, indices = model.kneighbors(target_weighted)
    
    return bmi_foods.iloc[indices[0]]

print('Weighted KNN model defined!')

### Test the Recommendation System

In [ ]:
# Example user
age = 25
weight = 70  # kg
height = 170  # cm
gender = 'Male'

# Calculate BMI
bmi = calculate_bmi(weight, height)
category, emoji, health_msg = get_bmi_category(bmi)
daily_calories, goal = calculate_daily_calories(weight, height, age, gender, category)

print('='*60)
print('📊 BMI RESULTS')
print('='*60)
print(f'Your BMI: {bmi} kg/m²')
print(f'Category: {emoji} {category}')
print(f'Health Message: {health_msg}')
print(f'Daily Calorie Target: {daily_calories} kcal')
print(f'Goal: {goal}')

In [ ]:
# Get recommendations
print('\n' + '='*60)
print('🍽️ FOOD RECOMMENDATIONS')
print('='*60)

for meal_time in ['Breakfast', 'Lunch', 'Dinner']:
    meal_emoji = {'Breakfast': '🌅', 'Lunch': '☀️', 'Dinner': '🌙'}[meal_time]
    print(f'\n{meal_emoji} {meal_time.upper()}')
    print('-'*40)
    
    recommendations = weighted_knn_recommend(data, category, meal_time, 3)
    
    if recommendations is not None:
        for idx, (_, food) in enumerate(recommendations.iterrows(), 1):
            flag = '🇪🇹' if food['Cuisine'] == 'Ethiopian' else '🌍'
            print(f'{idx}. {flag} {food["Name"]} ({food["Calories"]} cal)')
            print(f'   → {food["Reason"]}')

### Test Different BMI Categories

In [ ]:
# Test with different BMI categories
test_cases = [
    {'name': 'Underweight Person', 'weight': 50, 'height': 175, 'age': 22, 'gender': 'Female'},
    {'name': 'Normal Weight Person', 'weight': 70, 'height': 175, 'age': 30, 'gender': 'Male'},
    {'name': 'Overweight Person', 'weight': 85, 'height': 170, 'age': 35, 'gender': 'Male'},
    {'name': 'Obese Person', 'weight': 100, 'height': 165, 'age': 40, 'gender': 'Female'},
]

print('='*70)
print('TESTING DIFFERENT BMI CATEGORIES')
print('='*70)

for case in test_cases:
    bmi = calculate_bmi(case['weight'], case['height'])
    cat, emoji, msg = get_bmi_category(bmi)
    calories, goal = calculate_daily_calories(case['weight'], case['height'], case['age'], case['gender'], cat)
    
    print(f"\n{case['name']}:")
    print(f'  BMI: {bmi} → {emoji} {cat}')
    print(f'  Daily Calories: {calories} kcal | Goal: {goal}')
    
    rec = weighted_knn_recommend(data, cat, 'Lunch', 1)
    if rec is not None:
        food = rec.iloc[0]
        print(f'  Top Lunch: {food["Name"]} ({food["Calories"]} cal)')

# 9. Summary

## What We Did:
1. **Data Loading** - Loaded Ethiopian foods dataset
2. **Data Exploration** - Examined structure, statistics, distributions
3. **Data Cleaning** - Checked for missing values and duplicates
4. **Data Integration** - Demonstrated concat and merge operations
5. **Data Transformation** - Label encoding, feature scaling
6. **Data Reduction** - Removed unnecessary columns
7. **Visualization** - Created charts to understand patterns
8. **Machine Learning** - Built Weighted KNN recommendation model

## Technologies Used:
- Python, Pandas, NumPy
- Matplotlib, Seaborn
- Scikit-learn (KNN, StandardScaler, LabelEncoder)

## Key ML Concept:
**Weighted KNN** - Different weights for different BMI categories:
- Underweight → Prioritizes HIGH calories
- Obese → Prioritizes LOW calories, HIGH fiber